In [14]:
import pandas as pd
import quandl
import warnings
import math
import numpy as np
from sklearn import preprocessing,cross_validation,svm
from sklearn.linear_model import LinearRegression
warnings.filterwarnings("ignore")

In [15]:
quandl.ApiConfig.api_key = 'x1rx9cicvrEymET58JYh'
df = quandl.get_table('WIKI/PRICES')
print(df.head())

     ticker       date   open   high    low  close      volume  ex-dividend  \
None                                                                          
0         A 1999-11-18  45.50  50.00  40.00  44.00  44739900.0          0.0   
1         A 1999-11-19  42.94  43.00  39.81  40.38  10897100.0          0.0   
2         A 1999-11-22  41.31  44.00  40.06  44.00   4705200.0          0.0   
3         A 1999-11-23  42.50  43.63  40.25  40.25   4274400.0          0.0   
4         A 1999-11-24  40.13  41.94  40.00  41.06   3464400.0          0.0   

      split_ratio   adj_open   adj_high    adj_low  adj_close  adj_volume  
None                                                                       
0             1.0  31.041951  34.112034  27.289627  30.018590  44739900.0  
1             1.0  29.295415  29.336350  27.160002  27.548879  10897100.0  
2             1.0  28.183363  30.018590  27.330562  30.018590   4705200.0  
3             1.0  28.995229  29.766161  27.460188  27.460188   42

In [16]:
df =  df[['adj_open','adj_high','adj_low','adj_close','adj_volume']]
print(df.head())

       adj_open   adj_high    adj_low  adj_close  adj_volume
None                                                        
0     31.041951  34.112034  27.289627  30.018590  44739900.0
1     29.295415  29.336350  27.160002  27.548879  10897100.0
2     28.183363  30.018590  27.330562  30.018590   4705200.0
3     28.995229  29.766161  27.460188  27.460188   4274400.0
4     27.378319  28.613174  27.289627  28.012803   3464400.0


In [17]:
df['HL_PCT'] = (df['adj_high']-df['adj_close']) / df['adj_close']*100
df['PCT_change'] = (df['adj_close']-df['adj_open']) / df['adj_open']*100
df = df[['adj_close','HL_PCT','PCT_change','adj_volume']]
print(df.head())

      adj_close     HL_PCT  PCT_change  adj_volume
None                                              
0     30.018590  13.636364   -3.296703  44739900.0
1     27.548879   6.488361   -5.961807  10897100.0
2     30.018590   0.000000    6.511740   4705200.0
3     27.460188   8.397516   -5.294118   4274400.0
4     28.012803   2.143205    2.317468   3464400.0


In [18]:
forecast_col = 'adj_close'
df.fillna(-99999,inplace = True)
forecast_out = int(math.ceil(0.01*len(df)))
df['label'] = df[forecast_col].shift(-forecast_out)
df.dropna(inplace=True)


In [19]:
X = np.array(df.drop(['label'],1))
y = np.array(df['label'])
X = preprocessing.scale(X)


In [20]:
X_train,X_test,y_train,y_test = cross_validation.train_test_split(X,y,test_size = 0.2,random_state=123)

In [21]:
# Linear Regression
clf = LinearRegression()
clf.fit(X_train,y_train)
accuracy = clf.score(X_test,y_test)
print(accuracy)

0.627353265784


In [22]:
# SVM with linear Kernel
clf = svm.SVR()
clf.fit(X_train,y_train)
accuracy = clf.score(X_test,y_test)
print(accuracy)

0.274933998113


In [23]:
# SVM with polynomial kernel
clf = svm.SVR(kernel='poly')
clf.fit(X_train,y_train)
accuracy = clf.score(X_test,y_test)
print(accuracy)

0.413751844027
